In [1]:
import sys
sys.path.append("/home/sire/phd/srz228573/torchmd-net/torchmdnet/models")

In [2]:
from model import load_model
import numpy as np
import torch

In [3]:
#model = load_model("/home/sire/phd/srz228573/benchmarking_datasets/fone_output/torchmdnet/lips2/logs/epoch=209-val_loss=0.0165-test_loss=0.0897.ckpt", derivative=True)
model = load_model("/home/sire/phd/srz228573/torchmd-net/bench_data_sl/fone_output/torchmdnet/acac/300/logs/epoch=1689-val_loss=0.0092-test_loss=0.0388.ckpt", derivative = True)

RuntimeError: Error(s) in loading state_dict for TorchMD_Net:
	size mismatch for mean: copying a param with shape torch.Size([1, 1]) from checkpoint, the shape in current model is torch.Size([]).
	size mismatch for std: copying a param with shape torch.Size([1, 1]) from checkpoint, the shape in current model is torch.Size([]).

In [10]:
data_path = "/home/sire/phd/srz228573/torchmd-net/bench_data_sl/fone_dataset/mdsim_data/mace_data_sl/rmd17/npz_data/aspirin/val/nequip_npz.npz"

In [11]:
data = np.load(data_path)

In [12]:
data.files

['nuclear_charges',
 'coords',
 'energies',
 'forces',
 'old_indices',
 'old_energies',
 'old_forces']

In [20]:
data["pos"].shape

(1000, 83, 3)

In [32]:
sample_an = torch.tensor(data["atomic_numbers"], dtype=torch.int64)
sample_pos = torch.tensor(data["pos"][0], dtype = torch.float)
#sample_batch = torch.tensor(data["atomic_numbers"], dtype=torch.long)

In [35]:
data["energy"][0]

array([-357.76645385])

In [65]:
data["forces"][0].reshape(-1).tolist()

[0.140055,
 -0.072523,
 0.15614,
 0.292358,
 0.181314,
 0.649291,
 -0.04958,
 0.228713,
 -0.02782,
 0.105383,
 -0.180811,
 0.108561,
 -0.461815,
 -0.438055,
 0.072563,
 -0.11117,
 0.059924,
 -0.541595,
 -0.024586,
 0.259346,
 0.00498,
 0.487963,
 0.654707,
 0.033238,
 -0.249255,
 -0.274291,
 -0.325159,
 0.306945,
 -0.541175,
 -0.255979,
 0.170464,
 -0.451539,
 -0.338122,
 0.054783,
 0.11219,
 0.342195,
 0.106185,
 -0.131755,
 -0.22801,
 0.416102,
 0.134834,
 0.74536,
 0.867692,
 -0.015244,
 0.023338,
 0.227926,
 -0.429522,
 0.063006,
 0.018059,
 -0.187791,
 0.103225,
 0.498418,
 -0.497752,
 0.383707,
 -0.356182,
 -0.198849,
 -0.110827,
 -0.039437,
 -0.43186,
 -0.318474,
 0.026411,
 0.033006,
 -0.000774,
 0.162771,
 -0.026142,
 -0.159467,
 -0.059036,
 0.13238,
 -0.122671,
 0.315283,
 0.21945,
 0.006801,
 0.075723,
 0.059662,
 -0.894275,
 0.348508,
 0.156919,
 -0.095214,
 0.057062,
 -0.597611,
 0.233396,
 0.515035,
 0.123776,
 -0.525764,
 -1.20129,
 -1.230282,
 -0.901754,
 1.073646,
 0.4

In [33]:
sample_an.shape

torch.Size([83])

In [36]:
pred_e , pred_f = model(sample_an, sample_pos)

In [50]:
_.detach().numpy()

array([[ 0.45817465, -0.2956959 , -0.23315057],
       [ 0.06279428, -0.01850238,  0.04368188],
       [ 0.42838988,  0.13666423, -0.0648061 ],
       [-0.02978445,  0.49436283,  0.28060314],
       [-0.24293035, -0.4263128 , -0.0638736 ],
       [-0.06903883, -0.15259601, -0.14137742],
       [ 0.58165675, -0.5842903 , -0.41140145],
       [-0.14438964, -0.2826755 , -0.24725197],
       [ 0.21011165, -0.08785138,  0.37960175],
       [-0.00559542, -0.25560042,  0.03887326],
       [-0.160956  ,  0.09895778,  0.04044032],
       [-0.30424958,  0.06627703,  0.10894996],
       [ 0.19805554,  0.08746103, -0.17070633],
       [ 0.02268512, -0.42826727,  0.42648602],
       [-0.03301629, -0.00739603, -0.01840309],
       [ 1.0343399 ,  0.6759107 , -0.130508  ],
       [ 0.04414092, -0.0925744 ,  0.46376455],
       [-0.6248187 ,  0.43907487,  0.30372778],
       [ 0.18612924, -0.10546705, -0.02263986],
       [-0.02125129, -0.57210845, -0.3991884 ],
       [ 0.09043165, -0.00800273,  0.114

In [42]:
pred_e.detach().numpy()[0][0]

-358.01462

In [44]:
from tqdm import tqdm

In [54]:
y_true = []
y_pred = []
for i in tqdm(range(len(data["pos"]))):
    sample_an = torch.tensor(data["atomic_numbers"], dtype=torch.int64)
    sample_pos = torch.tensor(data["pos"][i], dtype = torch.float)
    pred_e , _ = model(sample_an, sample_pos)
    y_pred.append(pred_e.detach().numpy()[0][0])
    y_true.append(data["energy"][i][0])
    

100%|██████████| 1000/1000 [01:45<00:00,  9.49it/s]


In [55]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_pred)

0.2107833484191409

In [66]:
y_true = []
y_pred = []
for i in tqdm(range(len(data["pos"]))):
    sample_an = torch.tensor(data["atomic_numbers"], dtype=torch.int64)
    sample_pos = torch.tensor(data["pos"][i], dtype = torch.float)
    pred_e , pred_f = model(sample_an, sample_pos)
    y_pred.append(pred_f.detach().numpy().reshape(-1).tolist())
    y_true.append(data["forces"][i].reshape(-1).tolist())
    

100%|██████████| 1000/1000 [01:46<00:00,  9.37it/s]


In [67]:
mean_absolute_error(y_true, y_pred)

0.07813291870145879